In [1]:
import pandas as pd

In [2]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')

C:\Users\vinay\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\vinay\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
mnist.data.shape

(70000, 784)

In [4]:
col= []
for i in range(784):
    col.append(str(i))

In [5]:
y = pd.Series(mnist.target).astype('int')
X = pd.DataFrame(mnist.data,columns=col)


In [6]:
y.value_counts()

1    7877
7    7293
3    7141
2    6990
9    6958
0    6903
6    6876
8    6825
4    6824
5    6313
dtype: int64

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [9]:
import tensorflow as tf

In [10]:
X.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '774', '775', '776', '777', '778', '779', '780', '781', '782', '783'],
      dtype='object', length=784)

In [11]:
feat_cols = []

for col in X.columns:
    feat_cols.append(tf.feature_column.numeric_column(col))

In [72]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=5000,num_epochs=2000,shuffle=True)

In [73]:
classifier = tf.estimator.DNNClassifier(hidden_units=[5000,2000,500], n_classes=10,feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\vinay\\AppData\\Local\\Temp\\tmpub40ivlh', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000021486652978>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [74]:
classifier.train(input_fn=input_func,steps=50)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\vinay\AppData\Local\Temp\tmpub40ivlh\model.ckpt.
INFO:tensorflow:loss = 400802.34, step = 1
INFO:tensorflow:Saving checkpoints for 50 into C:\Users\vinay\AppData\Local\Temp\tmpub40ivlh\model.ckpt.
INFO:tensorflow:Loss for final step: 7628.5293.


In [75]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [76]:
note_predictions = list(classifier.predict(input_fn=pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\vinay\AppData\Local\Temp\tmpub40ivlh\model.ckpt-50
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [77]:
note_predictions[0]

{'logits': array([44.592678, 69.152466, 59.455673, 63.328777, 72.529526, 62.980972,
        67.46762 , 64.70954 , 72.02105 , 77.524185], dtype=float32),
 'probabilities': array([4.9342689e-15, 2.2877213e-04, 1.4064996e-08, 6.7640582e-07,
        6.6994880e-03, 4.7770254e-07, 4.2431097e-05, 2.6907016e-06,
        4.0291478e-03, 9.8899627e-01], dtype=float32),
 'class_ids': array([9], dtype=int64),
 'classes': array([b'9'], dtype=object)}

In [78]:
final_preds  = []
for pred in note_predictions:
    final_preds.append(pred['class_ids'][0])

In [79]:
from sklearn.metrics import classification_report,confusion_matrix

In [80]:
print(confusion_matrix(y_test,final_preds))

[[1906    1   32    7    0   60    8   12    0    0]
 [   0 2220   53   11    1   34   15   13    3    1]
 [   7   21 1755   44    2  100   39   76   21   17]
 [   0    1   84 1631    1  348    8   52   26   24]
 [   0   47   17    7  117   42    7   74   29 1753]
 [  14    1   24   62    1 1594   12   59   23   35]
 [   7    5  132   18    1  111 1800    8    3    1]
 [   7    6   42    9    1   28    1 1987    7   87]
 [   1    7  270  114    9 1064   30   39  447   91]
 [   6    6   10   11   11   37    0  208   29 1797]]


In [81]:
print(classification_report(y_test,final_preds))

              precision    recall  f1-score   support

           0       0.98      0.94      0.96      2026
           1       0.96      0.94      0.95      2351
           2       0.73      0.84      0.78      2082
           3       0.85      0.75      0.80      2175
           4       0.81      0.06      0.10      2093
           5       0.47      0.87      0.61      1825
           6       0.94      0.86      0.90      2086
           7       0.79      0.91      0.84      2175
           8       0.76      0.22      0.34      2072
           9       0.47      0.85      0.61      2115

   micro avg       0.73      0.73      0.73     21000
   macro avg       0.77      0.72      0.69     21000
weighted avg       0.78      0.73      0.69     21000



In [82]:
from sklearn.metrics import accuracy_score

In [83]:
print(accuracy_score(y_test,final_preds))

0.7263809523809523
